In [ ]:
# 导入所需的PyTorch和torchvision库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 检查是否有GPU可用，如果有则使用GPU，否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1. 数据预处理和加载
# 定义数据转换步骤，将图像转换为Tensor，并进行标准化处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为Tensor格式
    transforms.Normalize((0.5,), (0.5,))  # 标准化图像，使其均值为0.5，标准差为0.5
])

# 下载并加载训练集和测试集
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# 将数据集封装到DataLoader中，以便在训练和测试过程中批量加载
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)  # 训练集，批量大小64
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)   # 测试集，批量大小64

# 2. 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一层卷积：输入通道1（灰度图像），输出通道32，卷积核大小3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        # 第二层卷积：输入通道32，输出通道64，卷积核大小3x3
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # 最大池化层：核大小2x2，步长2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # 全连接层1：将特征图展平到64*7*7，输出128个特征
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # 全连接层2：输出10个特征（对应10个类别）
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # 第一层卷积、ReLU激活和池化
        x = self.pool(torch.relu(self.conv1(x)))
        # 第二层卷积、ReLU激活和池化
        x = self.pool(torch.relu(self.conv2(x)))
        # 将特征展平为1维向量，以便输入到全连接层
        x = x.view(-1, 64 * 7 * 7)
        # 第一个全连接层和ReLU激活
        x = torch.relu(self.fc1(x))
        # 第二个全连接层（输出层）
        x = self.fc2(x)
        return x

# 实例化CNN模型，并将模型移动到GPU（如果有）
model = CNN().to(device)

# 3. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数，适用于多分类任务
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用Adam优化器，学习率0.001

# 4. 训练模型
num_epochs = 5  # 训练轮次
for epoch in range(num_epochs):
    running_loss = 0.0  # 用于记录每个epoch的总损失
    for i, (images, labels) in enumerate(train_loader):
        # 将图像和标签移动到GPU
        images, labels = images.to(device), labels.to(device)

        # 将梯度清零，以便计算新的梯度
        optimizer.zero_grad()
        # 前向传播，计算模型输出
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播，计算梯度
        loss.backward()
        # 更新模型参数
        optimizer.step()
        
        # 累加损失值
        running_loss += loss.item()
        if i % 100 == 99:  # 每100个小批次打印一次损失
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

print('Finished Training')

# 5. 测试模型
correct = 0  # 记录正确分类的样本数
total = 0    # 记录测试集的总样本数

# 在测试过程中，不需要计算梯度，使用torch.no_grad()可以加速测试
with torch.no_grad():
    for images, labels in test_loader:
        # 将图像和标签移动到GPU
        images, labels = images.to(device), labels.to(device)

        # 获取模型输出
        outputs = model(images)
        # 获取最大值对应的类别
        _, predicted = torch.max(outputs.data, 1)
        # 累加正确分类的样本数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 打印模型的分类准确率
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')
